***GENERATED CODE FOR aiforgec PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intern', 'transformation_label': 'String Indexer'}], 'feature': 'Intern', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '326', 'mean': '', 'stddev': '', 'min': 'AYODEJI OLAWOYE', 'max': 'Zaryab Prasla', 'missing': '0', 'distinct': '18'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intern'}, {'feature_label': 'Intern', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Intern')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'GoalResponseLen', 'transformation_label': 'String Indexer'}], 'feature': 'GoalResponseLen', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '326', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'GoalResponseLen'}, {'feature_label': 'GoalResponseLen', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('GoalResponseLen')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternEmail', 'transformation_label': 'String Indexer'}], 'feature': 'InternEmail', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '326', 'mean': '', 'stddev': '', 'min': 'amirwaites@gmail.com', 'max': 'zaryabprasla35@gmail.com', 'missing': '0', 'distinct': '18'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InternEmail'}, {'feature_label': 'InternEmail', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternEmail')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternStartDate', 'transformation_label': 'String Indexer'}], 'feature': 'InternStartDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '326', 'mean': '', 'stddev': '', 'min': '1/19/2024', 'max': '8/8/2023', 'missing': '0', 'distinct': '15'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InternStartDate'}, {'feature_label': 'InternStartDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternStartDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternEndDate', 'transformation_label': 'String Indexer'}], 'feature': 'InternEndDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '1', 'mean': '', 'stddev': '', 'min': '00:00.0', 'max': '00:00.0', 'missing': '325', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InternEndDate'}, {'feature_label': 'InternEndDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternEndDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternTechGroupName', 'transformation_label': 'String Indexer'}], 'feature': 'InternTechGroupName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '326', 'mean': '', 'stddev': '', 'min': 'Data Warehousing and ETL', 'max': 'Qlik', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InternTechGroupName'}, {'feature_label': 'InternTechGroupName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternTechGroupName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternManager', 'transformation_label': 'String Indexer'}], 'feature': 'InternManager', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '326', 'mean': '', 'stddev': '', 'min': 'Ali Muwwakkil', 'max': 'Ali Muwwakkil', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'InternManager'}, {'feature_label': 'InternManager', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternManager')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PName', 'transformation_label': 'String Indexer'}], 'feature': 'PName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '320', 'mean': '', 'stddev': '', 'min': ' AI Forge: Pathway to Predictive Excellence', 'max': "Let's Go All In Analytics", 'missing': '6', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PName'}, {'feature_label': 'PName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'L_URL', 'transformation_label': 'String Indexer'}], 'feature': 'L_URL', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '320', 'mean': '', 'stddev': '', 'min': 'https://3.basecamp.com/3945211/buckets/24865175/todolists/6801626850', 'max': 'https://3.basecamp.com/3945211/buckets/24865175/todolists/7255296952', 'missing': '6', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'L_URL'}, {'feature_label': 'L_URL', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('L_URL')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intern_LastActivitySection', 'transformation_label': 'String Indexer'}], 'feature': 'Intern_LastActivitySection', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '325', 'mean': '', 'stddev': '', 'min': 'Automated Mock - Abel Kirui_04/26/2024_Ubisoft_Data Operations Manager Social Media',
                                                                                                                                                                                                                                                                                                      'max': 'Phase 2 - PowerBIVideos - Power BI  Video (Tell me about your Power BI Experience)', 'missing': '1', 'distinct': '17'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intern_LastActivitySectio...'}, {'feature_label': 'Intern_LastActivitySection', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Intern_LastActivitySection')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intern_MM_Mentor', 'transformation_label': 'String Indexer'}], 'feature': 'Intern_MM_Mentor', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '325', 'mean': '', 'stddev': '', 'min': 'ALEXANDER  OPIYO', 'max': 'hubert ndifusah', 'missing': '1', 'distinct': '13'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intern_MM_Mentor'}, {'feature_label': 'Intern_MM_Mentor', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Intern_MM_Mentor')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intern_MM_PlacementDate', 'transformation_label': 'String Indexer'}], 'feature': 'Intern_MM_PlacementDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '326', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intern_MM_PlacementDate'}, {'feature_label': 'Intern_MM_PlacementDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Intern_MM_PlacementDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intern_MM_WklyComments', 'transformation_label': 'String Indexer'}], 'feature': 'Intern_MM_WklyComments', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '326', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intern_MM_WklyComments'}, {'feature_label': 'Intern_MM_WklyComments', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Intern_MM_WklyComments')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Intern_MM_LastCommentDate', 'transformation_label': 'String Indexer'}], 'feature': 'Intern_MM_LastCommentDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '326', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Intern_MM_LastCommentDate'}, {'feature_label': 'Intern_MM_LastCommentDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Intern_MM_LastCommentDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Phase', 'transformation_label': 'String Indexer'}], 'feature': 'Phase', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '311', 'mean': '', 'stddev': '', 'min': 'Phase 1', 'max': 'Phase 2', 'missing': '15', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Phase'}, {'feature_label': 'Phase', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Phase')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CurrentSprintTask', 'transformation_label': 'String Indexer'}], 'feature': 'CurrentSprintTask', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '326', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CurrentSprintTask'}, {'feature_label': 'CurrentSprintTask', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CurrentSprintTask')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StudentProfile', 'transformation_label': 'String Indexer'}], 'feature': 'StudentProfile', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '326', 'mean': '', 'stddev': '', 'min': 'https://app.colaberry.com/app/ipbc/students/ipbc/33001', 'max': 'https://app.colaberry.com/app/ipbc/students/ipbc/43264', 'missing': '0', 'distinct': '18'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'StudentProfile'}, {'feature_label': 'StudentProfile', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StudentProfile')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProjectName_2', 'transformation_label': 'String Indexer'}], 'feature': 'ProjectName_2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '320', 'mean': '', 'stddev': '', 'min': ' AI Forge: Pathway to Predictive Excellence', 'max': "Let's Go All In Analytics", 'missing': '6', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProjectName_2'}, {'feature_label': 'ProjectName_2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProjectName_2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProjectKPI_Link', 'transformation_label': 'String Indexer'}], 'feature': 'ProjectKPI_Link', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '308', 'mean': '', 'stddev': '', 'min': 'https://app.colaberry.com/app/student/ProjMgmtByAdmin/1816', 'max': 'https://app.colaberry.com/app/student/ProjMgmtByAdmin/1956', 'missing': '18', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProjectKPI_Link'}, {'feature_label': 'ProjectKPI_Link', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProjectKPI_Link')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '326', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReportDate', 'transformation_label': 'String Indexer'}], 'feature': 'ReportDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '326', 'mean': '', 'stddev': '', 'min': '1/17/2023', 'max': '8/3/2022', 'missing': '0', 'distinct': '18'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReportDate'}, {'feature_label': 'ReportDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReportDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TopDISCscore', 'transformation_label': 'String Indexer'}], 'feature': 'TopDISCscore', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '326', 'mean': '', 'stddev': '', 'min': 'Compliance', 'max': 'Steadiness', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TopDISCscore'}, {'feature_label': 'TopDISCscore', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TopDISCscore')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LowDISCscore', 'transformation_label': 'String Indexer'}], 'feature': 'LowDISCscore', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '326', 'mean': '', 'stddev': '', 'min': 'Dominance', 'max': 'Steadiness', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LowDISCscore'}, {'feature_label': 'LowDISCscore', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LowDISCscore')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TopMotivator', 'transformation_label': 'String Indexer'}], 'feature': 'TopMotivator', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '326', 'mean': '', 'stddev': '', 'min': 'Aesthetic', 'max': 'Utilitarian', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TopMotivator'}, {'feature_label': 'TopMotivator', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TopMotivator')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LowMotivator', 'transformation_label': 'String Indexer'}], 'feature': 'LowMotivator', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '326', 'mean': '', 'stddev': '', 'min': 'Aesthetic', 'max': 'Utilitarian', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LowMotivator'}, {'feature_label': 'LowMotivator', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LowMotivator')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TopSkill', 'transformation_label': 'String Indexer'}], 'feature': 'TopSkill', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '326', 'mean': '', 'stddev': '', 'min': 'Flexibility', 'max': 'Teamwork', 'missing': '0', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TopSkill'}, {'feature_label': 'TopSkill', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TopSkill')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'LowSkill', 'transformation_label': 'String Indexer'}], 'feature': 'LowSkill', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '326', 'mean': '', 'stddev': '', 'min': 'Flexibility', 'max': 'Presenting', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'LowSkill'}, {'feature_label': 'LowSkill', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('LowSkill')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run aiforgecHooks.ipynb
try:
	#sourcePreExecutionHook()

	aiforgeactive = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/AI FORGE_Active_6-26-2024.csv', 'filename': 'AI FORGE_Active_6-26-2024.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(aiforgeactive)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run aiforgecHooks.ipynb
try:
	#transformationPreExecutionHook()

	aiforgecautofe = TransformationMain.run(aiforgeactive,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Intern", "transformation_label": "String Indexer"}], "feature": "Intern", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "326", "mean": "", "stddev": "", "min": "AYODEJI OLAWOYE", "max": "Zaryab Prasla", "missing": "0", "distinct": "18"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intern"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "DaysInInternship", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "315.27", "stddev": "208.44", "min": "2", "max": "1150", "missing": "0"}, "updatedLabel": "DaysInInternship"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ClassSignupsID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "13920.93", "stddev": "1708.01", "min": "10469", "max": "17002", "missing": "0"}, "updatedLabel": "ClassSignupsID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SurveySent", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "1.9", "stddev": "1.21", "min": "0", "max": "6", "missing": "0"}, "updatedLabel": "SurveySent"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "INTERNSHIP_SCORE", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "0.63", "stddev": "0.21", "min": "0.246153", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "INTERNSHIP_SCORE"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SurveysTaken", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "1.11", "stddev": "1.07", "min": "0", "max": "4", "missing": "0"}, "updatedLabel": "SurveysTaken"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SurveysGiven", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "1.21", "stddev": "0.93", "min": "0", "max": "4", "missing": "0"}, "updatedLabel": "SurveysGiven"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "QuestionsScored", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "15.64", "stddev": "12.11", "min": "0", "max": "52", "missing": "0"}, "updatedLabel": "QuestionsScored"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Work Ethic", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "0.71", "stddev": "0.41", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Work Ethic"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Availability", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "0.73", "stddev": "0.41", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Availability"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Team Player", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "0.73", "stddev": "0.41", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Team Player"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Quality of Work", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "0.73", "stddev": "0.41", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Quality of Work"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Trust_2", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "0.72", "stddev": "0.41", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Trust_2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Respectful", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "0.74", "stddev": "0.42", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Respectful"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SurveyScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "0.73", "stddev": "0.41", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SurveyScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BaseCamp_FinalScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "0.62", "stddev": "0.39", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BaseCamp_FinalScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_FinalScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "0.37", "stddev": "0.31", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "IPBC_FinalScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BaseCamp_Score", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "0.62", "stddev": "0.39", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "BaseCamp_Score"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Attendance", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "249", "mean": "2.6", "stddev": "1.33", "min": "1", "max": "6", "missing": "77"}, "updatedLabel": "Attendance"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BaseCampCommentScore", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "249", "mean": "2.6", "stddev": "1.33", "min": "1", "max": "6", "missing": "77"}, "updatedLabel": "BaseCampCommentScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "BaseCampTotalComments", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "249", "mean": "4.93", "stddev": "3.88", "min": "1", "max": "20", "missing": "77"}, "updatedLabel": "BaseCampTotalComments"}, {"transformationsData": [{"feature_label": "GoalResponseLen", "transformation_label": "String Indexer"}], "feature": "GoalResponseLen", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "326", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "GoalResponseLen"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_Score", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "325", "mean": "0.37", "stddev": "0.31", "min": "0.0", "max": "1.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "IPBC_Score"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InternID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "421.61", "stddev": "22.72", "min": "310", "max": "456", "missing": "0"}, "updatedLabel": "InternID"}, {"transformationsData": [{"feature_label": "InternEmail", "transformation_label": "String Indexer"}], "feature": "InternEmail", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "326", "mean": "", "stddev": "", "min": "amirwaites@gmail.com", "max": "zaryabprasla35@gmail.com", "missing": "0", "distinct": "18"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InternEmail"}, {"transformationsData": [{"feature_label": "InternStartDate", "transformation_label": "String Indexer"}], "feature": "InternStartDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "326", "mean": "", "stddev": "", "min": "1/19/2024", "max": "8/8/2023", "missing": "0", "distinct": "15"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InternStartDate"}, {"transformationsData": [{"feature_label": "InternEndDate", "transformation_label": "String Indexer"}], "feature": "InternEndDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "1", "mean": "", "stddev": "", "min": "00:00.0", "max": "00:00.0", "missing": "325", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InternEndDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InternTechGroupID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "3.02", "stddev": "0.4", "min": "1", "max": "10", "missing": "0"}, "updatedLabel": "InternTechGroupID"}, {"transformationsData": [{"feature_label": "InternTechGroupName", "transformation_label": "String Indexer"}], "feature": "InternTechGroupName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "326", "mean": "", "stddev": "", "min": "Data Warehousing and ETL", "max": "Qlik", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InternTechGroupName"}, {"transformationsData": [{"feature_label": "InternManager", "transformation_label": "String Indexer"}], "feature": "InternManager", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "326", "mean": "", "stddev": "", "min": "Ali Muwwakkil", "max": "Ali Muwwakkil", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "InternManager"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "internisactive", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "1.0", "stddev": "0.0", "min": "1", "max": "1", "missing": "0"}, "updatedLabel": "internisactive"}, {"transformationsData": [{"feature_label": "PName", "transformation_label": "String Indexer"}], "feature": "PName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "320", "mean": "", "stddev": "", "min": " AI Forge: Pathway to Predictive Excellence", "max": "Let's Go All In Analytics", "missing": "6", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PName"}, {"transformationsData": [{"feature_label": "L_URL", "transformation_label": "String Indexer"}], "feature": "L_URL", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "320", "mean": "", "stddev": "", "min": "https://3.basecamp.com/3945211/buckets/24865175/todolists/6801626850", "max": "https://3.basecamp.com/3945211/buckets/24865175/todolists/7255296952", "missing": "6", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "L_URL"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_WkOrder", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "22.94", "stddev": "19.99", "min": "1", "max": "78", "missing": "0"}, "updatedLabel": "Intern_MM_WkOrder"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_Wk", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "325", "mean": "21.27", "stddev": "12.03", "min": "1", "max": "52", "missing": "1"}, "updatedLabel": "Intern_MM_Wk"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_Yr", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "325", "mean": "2023.67", "stddev": "0.47", "min": "2023", "max": "2024", "missing": "1"}, "updatedLabel": "Intern_MM_Yr"}, {"transformationsData": [{"feature_label": "Intern_LastActivitySection", "transformation_label": "String Indexer"}], "feature": "Intern_LastActivitySection", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "325", "mean": "", "stddev": "", "min": "Automated Mock - Abel Kirui_04/26/2024_Ubisoft_Data Operations Manager Social Media", "max": "Phase 2 - PowerBIVideos - Power BI  Video (Tell me about your Power BI Experience)", "missing": "1", "distinct": "17"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intern_LastActivitySectio..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_HW", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "325", "mean": "0.07", "stddev": "0.41", "min": "0", "max": "4", "missing": "1"}, "updatedLabel": "Intern_MM_HW"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_Videos", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "325", "mean": "0.19", "stddev": "0.56", "min": "0", "max": "5", "missing": "1"}, "updatedLabel": "Intern_MM_Videos"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_VideosScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "325", "mean": "0.57", "stddev": "1.49", "min": "0.0", "max": "5.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_VideosScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_AutoInt", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "325", "mean": "0.26", "stddev": "0.82", "min": "0", "max": "6", "missing": "1"}, "updatedLabel": "Intern_MM_AutoInt"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_Phone", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "325", "mean": "0.06", "stddev": "0.28", "min": "0", "max": "2", "missing": "1"}, "updatedLabel": "Intern_MM_Phone"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_LoginPer", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "325", "mean": "0.62", "stddev": "0.3", "min": "0.0", "max": "1.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_LoginPer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_ActivityScore", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "325", "mean": "0.58", "stddev": "1.36", "min": "0", "max": "10", "missing": "1"}, "updatedLabel": "Intern_MM_ActivityScore"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_ParticipationPer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "325", "mean": "0.23", "stddev": "0.42", "min": "0", "max": "1", "missing": "1"}, "updatedLabel": "Intern_MM_ParticipationPe..."}, {"transformationsData": [{"feature_label": "Intern_MM_Mentor", "transformation_label": "String Indexer"}], "feature": "Intern_MM_Mentor", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "325", "mean": "", "stddev": "", "min": "ALEXANDER  OPIYO", "max": "hubert ndifusah", "missing": "1", "distinct": "13"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_Mentor"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_UserComments", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "325", "mean": "0.55", "stddev": "0.73", "min": "0", "max": "3", "missing": "1"}, "updatedLabel": "Intern_MM_UserComments"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_ActivityScoreChk", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "325", "mean": "0.23", "stddev": "0.42", "min": "0", "max": "1", "missing": "1"}, "updatedLabel": "Intern_MM_ActivityScoreCh..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_UserCommentsChk", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "325", "mean": "0.44", "stddev": "0.5", "min": "0", "max": "1", "missing": "1"}, "updatedLabel": "Intern_MM_UserCommentsChk"}, {"transformationsData": [{"feature_label": "Intern_MM_PlacementDate", "transformation_label": "String Indexer"}], "feature": "Intern_MM_PlacementDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "326", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_PlacementDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_JRPChk", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "325", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "1"}, "updatedLabel": "Intern_MM_JRPChk"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_SubChk", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "325", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "1"}, "updatedLabel": "Intern_MM_SubChk"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_AutoIntChk", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "325", "mean": "0.12", "stddev": "0.33", "min": "0", "max": "1", "missing": "1"}, "updatedLabel": "Intern_MM_AutoIntChk"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_PhoneChk", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "325", "mean": "0.05", "stddev": "0.22", "min": "0", "max": "1", "missing": "1"}, "updatedLabel": "Intern_MM_PhoneChk"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_MM_Wk_ActivityScore", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "310", "mean": "0.19", "stddev": "0.1", "min": "0.11", "max": "0.588571", "missing": "16"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_Wk_ActivityScor..."}, {"transformationsData": [{"feature_label": "Intern_MM_WklyComments", "transformation_label": "String Indexer"}], "feature": "Intern_MM_WklyComments", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "326", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_WklyComments"}, {"transformationsData": [{"feature_label": "Intern_MM_LastCommentDate", "transformation_label": "String Indexer"}], "feature": "Intern_MM_LastCommentDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "326", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Intern_MM_LastCommentDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_CC_Answer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "267", "mean": "2.34", "stddev": "5.49", "min": "0", "max": "28", "missing": "59"}, "updatedLabel": "Intern_CC_Answer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_CC_CorrectAnswer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "267", "mean": "0.01", "stddev": "0.09", "min": "0", "max": "1", "missing": "59"}, "updatedLabel": "Intern_CC_CorrectAnswer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_CC_VideosHosted", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "267", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "59"}, "updatedLabel": "Intern_CC_VideosHosted"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_CC_QualfyVideo", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "267", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "59"}, "updatedLabel": "Intern_CC_QualfyVideo"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_COE_Participation", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "267", "mean": "5.13", "stddev": "4.12", "min": "0", "max": "22", "missing": "59"}, "updatedLabel": "Intern_COE_Participation"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Intern_COE_ProjectUpdates", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "267", "mean": "0.01", "stddev": "0.14", "min": "0", "max": "2", "missing": "59"}, "updatedLabel": "Intern_COE_ProjectUpdates"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_Part1", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "325", "mean": "0.58", "stddev": "1.36", "min": "0", "max": "10", "missing": "1"}, "updatedLabel": "IPBC_Part1"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_Part2", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "1.38", "stddev": "3.0", "min": "0", "max": "10", "missing": "0"}, "updatedLabel": "IPBC_Part2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_Part3", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "3.83", "stddev": "3.32", "min": "0", "max": "10", "missing": "0"}, "updatedLabel": "IPBC_Part3"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "IPBC_Score_Pre_Total", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "325", "mean": "5.8", "stddev": "5.32", "min": "0", "max": "20", "missing": "1"}, "updatedLabel": "IPBC_Score_Pre_Total"}, {"transformationsData": [{"feature_label": "Phase", "transformation_label": "String Indexer"}], "feature": "Phase", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "311", "mean": "", "stddev": "", "min": "Phase 1", "max": "Phase 2", "missing": "15", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Phase"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InternLevelGroup", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "311", "mean": "2.32", "stddev": "0.47", "min": "2", "max": "3", "missing": "15"}, "updatedLabel": "InternLevelGroup"}, {"transformationsData": [{"feature_label": "CurrentSprintTask", "transformation_label": "String Indexer"}], "feature": "CurrentSprintTask", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "326", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CurrentSprintTask"}, {"transformationsData": [{"feature_label": "StudentProfile", "transformation_label": "String Indexer"}], "feature": "StudentProfile", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "326", "mean": "", "stddev": "", "min": "https://app.colaberry.com/app/ipbc/students/ipbc/33001", "max": "https://app.colaberry.com/app/ipbc/students/ipbc/43264", "missing": "0", "distinct": "18"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "StudentProfile"}, {"transformationsData": [{"feature_label": "ProjectName_2", "transformation_label": "String Indexer"}], "feature": "ProjectName_2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "320", "mean": "", "stddev": "", "min": " AI Forge: Pathway to Predictive Excellence", "max": "Let's Go All In Analytics", "missing": "6", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProjectName_2"}, {"transformationsData": [{"feature_label": "ProjectKPI_Link", "transformation_label": "String Indexer"}], "feature": "ProjectKPI_Link", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "308", "mean": "", "stddev": "", "min": "https://app.colaberry.com/app/student/ProjMgmtByAdmin/1816", "max": "https://app.colaberry.com/app/student/ProjMgmtByAdmin/1956", "missing": "18", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProjectKPI_Link"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "326", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Dominance", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "44.38", "stddev": "21.8", "min": "15", "max": "89", "missing": "0"}, "updatedLabel": "Dominance"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Influencer", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "51.1", "stddev": "16.23", "min": "8", "max": "82", "missing": "0"}, "updatedLabel": "Influencer"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Steadiness", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "61.49", "stddev": "22.18", "min": "18", "max": "93", "missing": "0"}, "updatedLabel": "Steadiness"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Compliance", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "54.74", "stddev": "16.92", "min": "38", "max": "100", "missing": "0"}, "updatedLabel": "Compliance"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Theoretical", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "56.29", "stddev": "12.88", "min": "37", "max": "78", "missing": "0"}, "updatedLabel": "Theoretical"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Utilitarian", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "50.77", "stddev": "13.92", "min": "27", "max": "78", "missing": "0"}, "updatedLabel": "Utilitarian"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Aesthetic", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "40.22", "stddev": "9.06", "min": "25", "max": "61", "missing": "0"}, "updatedLabel": "Aesthetic"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Social", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "40.19", "stddev": "19.14", "min": "8", "max": "70", "missing": "0"}, "updatedLabel": "Social"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Individualistic", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "42.22", "stddev": "10.46", "min": "26", "max": "70", "missing": "0"}, "updatedLabel": "Individualistic"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Traditional", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "39.87", "stddev": "12.64", "min": "17", "max": "70", "missing": "0"}, "updatedLabel": "Traditional"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Flexibility", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "30.53", "stddev": "31.3", "min": "1.5", "max": "84.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Flexibility"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "FuturisticThinking", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "11.18", "stddev": "17.42", "min": "0.0", "max": "60.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "FuturisticThinking"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "GoalOrientation", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "30.57", "stddev": "34.78", "min": "4.3", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "GoalOrientation"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "InterpersonalSkills", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "35.63", "stddev": "40.01", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "InterpersonalSkills"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Leadership", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "27.25", "stddev": "34.21", "min": "0.0", "max": "83.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Leadership"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Negotiation", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "27.13", "stddev": "32.91", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Negotiation"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PersonalResponsibility", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "29.11", "stddev": "31.29", "min": "3.4", "max": "81.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PersonalResponsibility"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Persuasion", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "19.09", "stddev": "31.78", "min": "0.0", "max": "83.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Persuasion"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PlanningOrganizing", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "30.48", "stddev": "35.64", "min": "2.7", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PlanningOrganizing"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Presenting", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "27.48", "stddev": "34.16", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Presenting"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Teamwork", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "32.9", "stddev": "37.5", "min": "2.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Teamwork"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "WrittenCommunication", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "24.82", "stddev": "32.52", "min": "0.0", "max": "100.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "WrittenCommunication"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "UnderstandingOthers", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "32.61", "stddev": "32.89", "min": "0.0", "max": "90.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "UnderstandingOthers"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "PracticalThinking", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "34.19", "stddev": "34.3", "min": "2.2", "max": "88.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "PracticalThinking"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SystemsJudgment", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "29.64", "stddev": "30.26", "min": "4.2", "max": "84.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SystemsJudgment"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SenseOfSelf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "30.31", "stddev": "32.88", "min": "3.2", "max": "82.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SenseOfSelf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "RoleAwareness", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "30.41", "stddev": "35.56", "min": "1.0", "max": "88.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "RoleAwareness"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SelfDirection", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "326", "mean": "28.57", "stddev": "30.3", "min": "3.6", "max": "76.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "SelfDirection"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "UnderstandingOthersBias", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "326", "mean": "-0.91", "stddev": "0.29", "min": "-1", "max": "0", "missing": "0"}, "updatedLabel": "UnderstandingOthersBias"}, {"transformationsData": [{"feature_label": "ReportDate", "transformation_label": "String Indexer"}], "feature": "ReportDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "326", "mean": "", "stddev": "", "min": "1/17/2023", "max": "8/3/2022", "missing": "0", "distinct": "18"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReportDate"}, {"transformationsData": [{"feature_label": "TopDISCscore", "transformation_label": "String Indexer"}], "feature": "TopDISCscore", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "326", "mean": "", "stddev": "", "min": "Compliance", "max": "Steadiness", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TopDISCscore"}, {"transformationsData": [{"feature_label": "LowDISCscore", "transformation_label": "String Indexer"}], "feature": "LowDISCscore", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "326", "mean": "", "stddev": "", "min": "Dominance", "max": "Steadiness", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LowDISCscore"}, {"transformationsData": [{"feature_label": "TopMotivator", "transformation_label": "String Indexer"}], "feature": "TopMotivator", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "326", "mean": "", "stddev": "", "min": "Aesthetic", "max": "Utilitarian", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TopMotivator"}, {"transformationsData": [{"feature_label": "LowMotivator", "transformation_label": "String Indexer"}], "feature": "LowMotivator", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "326", "mean": "", "stddev": "", "min": "Aesthetic", "max": "Utilitarian", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LowMotivator"}, {"transformationsData": [{"feature_label": "TopSkill", "transformation_label": "String Indexer"}], "feature": "TopSkill", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "326", "mean": "", "stddev": "", "min": "Flexibility", "max": "Teamwork", "missing": "0", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TopSkill"}, {"transformationsData": [{"feature_label": "LowSkill", "transformation_label": "String Indexer"}], "feature": "LowSkill", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "326", "mean": "", "stddev": "", "min": "Flexibility", "max": "Presenting", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "LowSkill"}]}))

	#transformationPostExecutionHook(aiforgecautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run aiforgecHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(aiforgecautofe, ["DaysInInternship", "ClassSignupsID", "SurveySent", "INTERNSHIP_SCORE", "SurveysTaken", "SurveysGiven", "QuestionsScored", "Work Ethic", "Availability", "Team Player", "Quality of Work", "Trust_2", "Respectful", "SurveyScore", "BaseCamp_FinalScore", "IPBC_FinalScore", "BaseCamp_Score", "Attendance", "BaseCampCommentScore", "BaseCampTotalComments", "IPBC_Score", "InternID", "InternTechGroupID", "Intern_MM_WkOrder", "Intern_MM_Wk", "Intern_MM_Yr", "Intern_MM_HW", "Intern_MM_Videos", "Intern_MM_VideosScore", "Intern_MM_AutoInt", "Intern_MM_Phone", "Intern_MM_LoginPer", "Intern_MM_ActivityScore", "Intern_MM_ParticipationPer", "Intern_MM_UserComments", "Intern_MM_ActivityScoreChk", "Intern_MM_UserCommentsChk", "Intern_MM_JRPChk", "Intern_MM_SubChk", "Intern_MM_AutoIntChk", "Intern_MM_PhoneChk", "Intern_MM_Wk_ActivityScore", "Intern_CC_Answer", "Intern_CC_CorrectAnswer", "Intern_CC_VideosHosted", "Intern_CC_QualfyVideo", "Intern_COE_Participation", "Intern_COE_ProjectUpdates", "IPBC_Part1", "IPBC_Part2", "IPBC_Part3", "IPBC_Score_Pre_Total", "InternLevelGroup", "Dominance", "Influencer", "Steadiness", "Compliance", "Theoretical", "Utilitarian", "Aesthetic", "Social", "Individualistic", "Traditional", "Flexibility", "FuturisticThinking", "GoalOrientation", "InterpersonalSkills", "Leadership", "Negotiation", "PersonalResponsibility", "Persuasion", "PlanningOrganizing", "Presenting", "Teamwork", "WrittenCommunication", "UnderstandingOthers", "PracticalThinking", "SystemsJudgment", "SenseOfSelf", "RoleAwareness", "SelfDirection", "UnderstandingOthersBias", "Intern_stringindexer", "GoalResponseLen_stringindexer", "InternEmail_stringindexer", "InternStartDate_stringindexer", "InternEndDate_stringindexer", "InternTechGroupName_stringindexer", "InternManager_stringindexer", "PName_stringindexer", "L_URL_stringindexer", "Intern_LastActivitySection_stringindexer", "Intern_MM_Mentor_stringindexer", "Intern_MM_PlacementDate_stringindexer", "Intern_MM_WklyComments_stringindexer", "Intern_MM_LastCommentDate_stringindexer", "Phase_stringindexer", "CurrentSprintTask_stringindexer", "StudentProfile_stringindexer", "ProjectName_2_stringindexer", "ProjectKPI_Link_stringindexer", "Gender_stringindexer", "ReportDate_stringindexer", "TopDISCscore_stringindexer", "LowDISCscore_stringindexer", "TopMotivator_stringindexer", "LowMotivator_stringindexer", "TopSkill_stringindexer", "LowSkill_stringindexer"], "internisactive")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

